In [1]:
#REMAP ALL OF THIS TO BE SIZE-AGNOSTIC, AND HAVE EACH SEGMENT BE MAX 0.5 MICRON
#OTHERWISE MESHING IS TOO COSTLY, AND IT'S ANYWAY NOT REAL (HEIDLBERG2>=0.6UM)

In [2]:
import numpy as np
import warnings; warnings.filterwarnings("ignore")
from ipywidgets import interact, IntSlider
import bezier

from phidl import Device,Layer, LayerSet, Port
from scipy.spatial import cKDTree
import phidl.geometry as pg
import phidl.routing as pr 
from scipy.spatial.distance import cdist
#if you have qt, use this
%matplotlib qt
from phidl import quickplot2 as qp
import networkx as nx
#and comment this
#from phidl.quickplotter import quickplot as qp
import matplotlib.pyplot as plt
from scipy import interpolate
import phidl.path as pp
from scipy.signal import savgol_filter



def get_norm(x,y):

    dy=np.gradient(y)
    dx=np.gradient(x)
    norm=np.array([dy,-dx])
    dn=norm/np.sqrt(np.sum(norm**2,axis=0))
    return dn
def thicken_to_poly(x,y,thick,flip_norm=False):
    norm_x,norm_y=get_norm(x,y)
    x_in,y_in=x+thick/2.*norm_x,y+thick/2.*norm_y
    x_out,y_out=x-thick/2.*norm_x,y-thick/2.*norm_y
    
    x_in=np.hstack((x_in[:-1],x_in[0]))
    y_in=np.hstack((y_in[:-1],y_in[0]))
    x_out=np.hstack((x_out[:-1],x_out[0]))
    y_out=np.hstack((y_out[:-1],y_out[0]))

    x=np.hstack((x_in,x_out[::-1]))
    y=np.hstack((y_in,y_out[::-1])) #reverse order to close polygon
    return np.vstack((x,y))

C:\ProgramData\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\ProgramData\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:

def snap_between_poly(poly1,poly2,idx1,idx2,ang_1,ang_2,mag1,mag2,
                      thickness,n_bezier=200,n_pts_1=3,n_pts_2=3,
                      n_ondul=0,amp_ondul=0):

    span_1=np.arange(idx1-n_pts_1,idx1+n_pts_1+1)%(len(poly1))
    span_2=np.arange(idx2-n_pts_2,idx2+n_pts_2+1)%(len(poly2))
    cap_1=poly1[span_1.astype(int),:]
    cap_2=poly2[span_2.astype(int),:]
    
    norm_1=get_norm(cap_1[:,0],cap_1[:,1])[:,n_pts_1]#[n_pts_1,:]#middle of the cap, i.e our pt1 point
    norm_2=get_norm(cap_2[:,0],cap_2[:,1])[:,n_pts_2]#[n_pts_2,:]#middle of the cap, i.e our pt1 point
    
    pt1=poly1[idx1]
    pt2=poly2[idx2]

    n_bits=2

    ang_1=ang_1/180*np.pi
    ang_2=ang_2/180*np.pi
    bitx1=[pt1[0]]#+mag1/float(n_bits)*norm_1[0]
    bitx2=[pt2[0]]
    bity1=[pt1[1]]
    bity2=[pt2[1]]
    for n in range(n_bits):
        n1_x=norm_1[0]*np.cos(n*ang_1)+norm_1[1]*np.sin(n*ang_1)
        n1_y=norm_1[1]*np.cos(n*ang_1)-norm_1[0]*np.sin(n*ang_1)
        
        n2_x=norm_2[0]*np.cos(n*ang_2)+norm_2[1]*np.sin(n*ang_2)
        n2_y=norm_2[1]*np.cos(n*ang_2)-norm_2[0]*np.sin(n*ang_2)

        bitx1.append(bitx1[-1]+mag1/float(n_bits)*n1_x)
        bitx2.append(bitx2[-1]+mag2/float(n_bits)*n2_x)

        bity1.append(bity1[-1]+mag1/float(n_bits)*n1_y)
        bity2.append(bity2[-1]+mag2/float(n_bits)*n2_y)
    bitx2=bitx2[::-1]
    bity2=bity2[::-1]
    #bitx1=[pt1[0]+x*norm_1[0] for x in np.linspace(0,mag1,n_bits)]
    #bitx2=[pt2[0]+x*norm_2[0] for x in np.linspace(mag2,0,n_bits)]
    #bity1=[pt1[1]+x*norm_1[1] for x in np.linspace(0,mag1,n_bits)]
    #bity2=[pt2[1]+x*norm_2[1] for x in np.linspace(mag2,0,n_bits)]
       
    #turn into closed polygon   
    nodes1 = np.asfortranarray([
         bitx1+bitx2,
         bity1+bity2])
    curve1 = bezier.Curve(nodes1, degree=2*n_bits+1)
    
    #upper_edge
    #n_bezier=25
    thick_bez=thickness
    x_bez,y_bez= curve1.evaluate_multi(np.linspace(0,1,n_bezier+2))
    x_bez=x_bez[1:-1]
    y_bez=y_bez[1:-1]
    norm_bez=get_norm(x_bez,y_bez)
    
    #make it wavey
    theta=np.linspace(0,n_ondul*np.pi,len(x_bez))
    x_bez+=norm_bez[0]*np.sin(theta)*amp_ondul #make the latch ondulated
    y_bez+=norm_bez[1]*np.sin(theta)*amp_ondul
    norm_bez=get_norm(x_bez,y_bez)
    ret=np.hstack((x_bez,y_bez))
    
    pt1=cap_1[0,:] #first pt1 of cap
    pt2=cap_2[-1,:]
    bez_ctrl0=np.vstack( (x_bez+thick_bez/2.*norm_bez[0],y_bez+thick_bez/2.*norm_bez[1]))
    curve=bezier.Curve(np.asfortranarray(np.hstack((pt1.reshape(2,1),bez_ctrl0,pt2.reshape(2,1)))),n_bezier+1)
    upper_edge=curve.evaluate_multi(np.linspace(0,1,200))
    
    #lower_edge
    pt1=cap_2[0,:] #first pt1 of cap
    pt2=cap_1[-1,:]
    bez_ctrl1=np.vstack( (x_bez-thick_bez/2.*norm_bez[0],y_bez-thick_bez/2.*norm_bez[1]))[:,::-1]
    curve=bezier.Curve(np.asfortranarray(np.hstack((pt1.reshape(2,1),bez_ctrl1,pt2.reshape(2,1)))),n_bezier+1)
    lower_edge=curve.evaluate_multi(np.linspace(0,1,200))
   #print(norm_bez[0])




    xy=np.vstack( (upper_edge.T,cap_2[::-1,:],lower_edge.T,cap_1[::-1,:]))
    
    #route a midline between both edges:

    curve=bezier.Curve(np.asfortranarray(np.hstack((cap_1[n_pts_1,:].reshape(2,1),np.vstack((x_bez,y_bez)),cap_2[n_pts_2,:].reshape(2,1)))),n_bezier+1)
    midline=curve.evaluate_multi(np.linspace(0,1,200)) #this is useful for setting the gold path
    
    if n_ondul!=0 and amp_ondul!=0: 

        c=midline.copy()
        arc_length_t=np.cumsum(np.hstack((0,np.sqrt(np.diff(c[0])**2+np.diff(c[1])**2))))
        arc_length_t/=arc_length_t[-1]
        t=arc_length_t*n_ondul*np.pi
        th_ond=0.5*(np.tanh((t-t[int(0.2*len(c[0]))]))+np.tanh((-(t-t[-1]+t[int(0.2*len(c[0]))]))))
        shift=np.sin(t)*th_ond 
        norm_x,norm_y=get_norm(c[0],c[1])
        cx,cy=c[0]+amp_ondul*norm_x*shift,c[1]+amp_ondul*norm_y*shift
        
        norm_ond_x,norm_ond_y=get_norm(cx,cy)
        upper_edge[0]=th_ond*(cx+norm_ond_x*thickness/2.)+(1-th_ond)*upper_edge[0] #only modify where ond is strng
        upper_edge[1]=th_ond*(cy+norm_ond_y*thickness/2.)+(1-th_ond)*upper_edge[1] #only modify where ond is strng
        

        low_idx=len(c[0])-np.arange(len(c[0]))-1
        lower_edge[0][low_idx]=th_ond*(cx-norm_ond_x*thickness/2.)+(1-th_ond)*lower_edge[0][low_idx]
        lower_edge[1][low_idx]=th_ond*(cy-norm_ond_y*thickness/2.)+(1-th_ond)*lower_edge[1][low_idx]

        
        xy=np.vstack( (upper_edge.T,cap_2[::-1,:],lower_edge.T,cap_1[::-1,:]))
        midline=np.vstack((cx,cy))
        
        mid,u,l=midline,upper_edge,lower_edge
        dist_l=np.sqrt(np.sum((mid.T-u.T)**2,axis=1))
        dist_u=np.sqrt(np.sum((mid.T-l.T[::-1,::])**2,axis=1))
     

        idx=np.asarray(np.where((np.abs(dist_l-dist_u)>0.1*thickness)&(np.abs(dist_l-dist_u)<thickness)))
        idx=idx[(idx>5)&(idx<=len(dist_l)-6)] #make sure we don't move the hooks (end points)

        mid_cent=mid.copy()
        mid_cent[:,idx]=(u[:,idx]+l[:,-idx-1])/2.
        midline=mid_cent
        
    return xy,midline

def draw_device(a_device):
    a_device
    polys = a_device.get_polygons()

    from PIL import Image, ImageDraw

    #This is messy. could be automated but I was lazy.
    #I upscale the quality and cast it to pixels, because
    #I need a good way to import it into a meshing software for simulation
    #And I do image analysis to reconstruct the mesh..

    img = Image.new('1', (5000, 5000))
    drw = ImageDraw.Draw(img, '1')
    
    span_x,span_y=np.diff(a_device.get_bounding_box().T)[:]
    scale=np.max((span_x,span_y))/(5000)
    
    shift_x,shift_y=(a_device.get_bounding_box()/scale)[0,:]
    
    for p in polys:
        p=np.vstack((p,p[0]))
        #p[:,0]-=np.min(p[:,0])
        #p[:,1]-=np.min(p[:,1])
        
        p=p/(1.5*scale) #scale
        
        #dimx,dimy=int(np.max(p[:,0])),int(np.max(p[:,1]))
        p[:,0]-=shift_x
        p[:,1]-=shift_y

        drw.polygon(list(map(tuple,p)), (1)) 
    del drw

    return img,scale,shift_x,shift_y   

In [4]:
def squircle(block_rad,x_c,y_c,p=3):
    squircle=Device()
    theta=np.linspace(0,2*np.pi,5000)
    r=block_rad/(np.abs(np.cos(theta))**p+np.abs(np.sin(theta))**p)**(1./p)
    x=r*np.cos(theta)
    y=r*np.sin(theta)
    #squircle.add_polygon(np.vstack((x,y)))
    return x,y

# rads=np.linspace(5500,8900,14)
# ps=np.linspace(3,7,14)
# for sq_rad,p in zip(rads,ps):
#     xs_sq=np.vstack((squircle(sq_rad,0,0,p=p))).T
#     f.add_polygon(thicken_to_poly(xs_sq[:,0],xs_sq[:,1],50))

In [5]:
def chop_ring(ring_poly,n_long,n_lat):
    '''ringpoly is an N-2 ring polygon (wavey loop)'''
    if ring_poly.shape[0]==2:
        ring_poly=ring_poly.T
    l=ring_poly.shape[0]
    outer_xy=ring_poly[:l//2,:]
    inner_xy=ring_poly[l//2:,:][::-1,:]
    l=len(outer_xy)
    steps_long=l//n_long
    
    
    t=np.linspace(0,1,n_lat+3)[1:-1] #for latitude control
    if n_lat==2:
        t=np.linspace(0.4,0.6,n_lat+3)[1:-1]
    chopped_device=SmartDevice()
    smartblocks=[]
    
    for i in range(n_long):
        begin,stop=i*steps_long,((i+1)*steps_long+1)#+1 needed on the stop to stick 
        idx=np.arange(begin,stop)%l
        inside,outside=inner_xy[idx,:],outer_xy[idx,:]
        for j in range(n_lat):

            out=outside+t[j]*(inside-outside)
            ins=outside+t[j+1]*(inside-outside)
            #mid_edge=0.5*(out[0,:]+ins[0,:])
            #mid_edge_2=0.5*(out[-1,:]+ins[-1,:])
            #block=np.vstack( (mid_edge,out,mid_edge_2,ins[::-1,:]))
            block=np.vstack((out,ins[::-1,:]))
            #hooks=[0,int(np.floor(len(out)/2))+1,len(out)+1,len(out)+1+int(np.ceil(len(ins)/2))]
            hooks=[0] # more hooks = overlapping hooks when tiling
            smartblocks.append(SmartBlock('ringblock_'+str(n_long)+'_'+str(n_lat),block,hooks))

    return smartblocks

def smart_latch(pol1,pol2,idx1,idx2,ang_1,ang_2,mag1,mag2,thickness,n_bezier,n_pts_1,n_pts_2,n_ondul=0,amp_ondul=0):
    polygon,midline=snap_between_poly(pol1,pol2,idx1,idx2,ang_1,ang_2,mag1,mag2,thickness,n_bezier,n_pts_1,n_pts_2,n_ondul,amp_ondul)
    hook_idx=[-n_pts_1-1,(len(polygon)-(2*(n_pts_1+n_pts_2+1)))//2+n_pts_2] #these are the middle point of each latch's caps
    return SmartBlock('latch',polygon,hook_idx=hook_idx,route_line=midline)

def smart_electrode(radius,x_pos,y_pos,resolution):
    t=np.linspace(0,2*np.pi,resolution)
    x=np.hstack((0,radius*np.cos(t)))+ x_pos #put a point in the center for hook
    y=np.hstack((0,radius*np.sin(t)))+ y_pos
    return SmartBlock('electrode',polygon=np.vstack((x,y)).T,hook_idx=[0])
    

In [6]:
class SmartBlock:
    def __init__(self,block_type,polygon,hook_idx,route_line=None):
        if not ((block_type in ['electrode','pad','latch','pad_to_open']) or ('ringblock' in block_type)) :
            raise ValueError('The allowed block types are electrode, pad, latch and ringblock and pad_to_open')
        self.block_type=block_type
        if polygon.shape[1]!=2:
            polygon=polygon.T
        self.polygon=polygon
        self.hook_idx=hook_idx
        self.normals=get_norm(polygon[:,0],polygon[:,1])[:,hook_idx].T
        self.route_line=route_line

In [7]:
class SmartDevice(Device):
    def __init__(self):
        '''initializes everything the Device needs, and a few helper things such as the graph and the smart blocks
        that help with routing gold'''
        Device.__init__(self)
        
        self.graph=nx.Graph()
        self.smart_blocks=[]
        self.all_hook_idx=[]
        self.all_hook_pos=[]
    def add_smart_block(self,smart_block_instance,add_to_design=True,layer=0):
        '''adds a smart block. These blocks have helping hooks to accelerate gold-path routing
        Sometimes, these blocks are only helpers, or chopped-up larger structures. As such, it is desirable sometimes
        to only use these blocks as routing helpers, but use only use the large, non-chopped structure for the design
        to save on the number of polygons. add_to_design tells whether to add these blocks to the overall design.'''        
        self.smart_blocks.append(smart_block_instance)
        if add_to_design:
            super(SmartDevice,self).add_polygon(smart_block_instance.polygon,layer=layer) #tells SmartDevice to go to parent (Device) and find addpoly
        self.all_hook_idx.append(smart_block_instance.hook_idx)
        self.all_hook_pos.append(smart_block_instance.polygon[smart_block_instance.hook_idx,:])
        
    def route_gold_lines(self,hook_tolerance=100,el_to_hook_tolerance=10,gold_thickness=5.):#microns
        def who_connects_who_closest(l1,l2,tol):
            l1=np.asarray(l1)
            l2=np.asarray(l2)
            xy1=self.linearized_hook_pos[l1]
            xy2=self.linearized_hook_pos[l2]

            dists=np.min(cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2]),axis=1)
            who=l2[np.argmin(cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2]),axis=1)]
            who=who[dists<tol] # distance has to be smaller than 100 microns (avoids pad-latches to connect to ring. later on..fix)
            return list(map(tuple,np.vstack( (l1[dists<tol],who)).T))
        def connect_closest(l1,l2,tol):
            l1=np.asarray(l1)
            l2=np.asarray(l2)
            xy1=self.linearized_hook_pos[l1]
            xy2=self.linearized_hook_pos[l2]

            dists=np.min(cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2]),axis=1)
            who=l2[np.argmin(cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2]),axis=1)]
             # distance has to be smaller than 100 microns (avoids pad-latches to connect to ring. later on..fix)
            return list(map(tuple,np.vstack( (l1,who)).T))
        def who_connects_who_radius(l1,l2):
            l1=np.asarray(l1)
            l2=np.asarray(l2)
            xy1=self.linearized_hook_pos[l1]
            xy2=self.linearized_hook_pos[l2]

            dists=cdist(self.linearized_hook_pos[l1],self.linearized_hook_pos[l2])
            kl1=l1[np.where(dists<2*hook_tolerance)[0]]
            kl2=l2[np.where(dists<2*hook_tolerance)[1]]
            return list(map(tuple,np.vstack((kl1,kl2)).T))
        def construct_path(hook_path,resolution):
            '''resolution of path in microns'''
            xy=[]
            cum_length_per_latch=[] #useful to thicken gold lines differently
            path_by_block=self.linearized_block_types[hook_path]
            for i in range(1,len(hook_path)):
                if path_by_block[i]==path_by_block[i-1]=='latch':
                    #begin_of_latch
                    cum_length_per_latch.append(   np.sum(np.sqrt(np.sum(np.diff(np.concatenate(xy),axis=0)**2,axis=1))   ))
                    
                    block_number=self.linear_to_block_index[hook_path[i]]
                    midline=self.smart_blocks[block_number].route_line.T
                    if self.linearized_hook_indices[hook_path[i-1]]>self.linearized_hook_indices[hook_path[i]]: #path is flipped wrt route line
                        midline=midline[::-1,:]                                                             
                    xy.append(midline)
                    #end_of_latch
                    cum_length_per_latch.append(   np.sum(np.sqrt(np.sum(np.diff(np.concatenate(xy),axis=0)**2,axis=1))   ))

                else:
                    xy_end=self.linearized_hook_pos[hook_path[i]]
                    xy_start=self.linearized_hook_pos[hook_path[i-1]]
                    dist=np.sum((xy_end-xy_start)**2)
                    t=np.linspace(0,1,resolution)
                    xy.append((xy_start.reshape(2,1)+t*(xy_end.reshape(2,1)-xy_start.reshape(2,1))).T) #interpolate path with some res.

            x,y=np.concatenate(xy).T 
            keep=np.where(np.abs(np.diff(x)+np.diff(y))>1e-2)[0]
            if len(x)-1 not in keep:
                keep=np.hstack((keep,len(x)-1)) #add back the pad, which is the last element
            x,y=x[keep],y[keep]   
            return x,y,cum_length_per_latch
       
        def group_consecutives(vals, step=1):
            """Return list of consecutive lists of numbers from vals (number list)."""
            run = []
            result = [run]
            expect = None
            for v in vals:
                if (v == expect) or (expect is None):
                    run.append(v)
                else:
                    run = [v]
                    result.append(run)
                expect = v + step
            return result
        def thicken_path(x,y,start_w=gold_thickness/2.,stop_w=gold_thickness/2.,smooth=0.05,frac=0.2):
            '''smooth is the slope of tanh transition, frac is when to jump to stop_w'''
            t=np.arange(len(x))
            offset,div=len(x)*frac,len(x)*smooth
            transition = 0.5*(np.tanh( (t-offset)/div)+1)*(stop_w-start_w)+start_w
            xn,yn=get_norm(x,y)*transition
            xpol=np.hstack( (x+xn,(x-xn)[::-1]))
            ypol=np.hstack( (y+yn,(y-yn)[::-1]))
            return np.vstack((xpol,ypol)).T
        def thicken_path_modulate(x,y,amps,percs,steep):
            def tanh_smooth(amps,percs,steep,support):
                x=support
                percs=np.asarray(percs)*len(x)
                steep=np.asarray(steep)*len(x)
                module=np.ones(len(x))*amps[0] # starting thickness
                for idx,(a,p,s) in enumerate(zip(np.diff(amps),percs,steep)):

                    module+=0.5*(np.tanh( (x-p)/s)+1)*a
                return module
            transition=tanh_smooth(amps,percs,steep,np.arange(len(x)))
            xn,yn=get_norm(x,y)*transition
            xpol=np.hstack( (x+xn,(x-xn)[::-1]))
            ypol=np.hstack( (y+yn,(y-yn)[::-1]))
            return np.vstack((xpol,ypol)).T
        self.linearized_hook_indices=(np.cumsum(np.concatenate([np.ones(len(el)) for el in self.all_hook_idx]))-1).astype(int)
        self.linearized_block_types=np.concatenate([len(s.hook_idx)*(s.block_type,) for s in self.smart_blocks])
        self.linear_to_block_index=np.concatenate([i*np.ones(len(el.hook_idx)) for i,el in enumerate(self.smart_blocks)]).astype(int)
        self.linearized_hook_pos=np.concatenate(self.all_hook_pos)
        ##---We make a square grid from the blocks, and disallow diagonal walks, so there's no possible path-crossings on this grid
        self.connected_hooks=[]
        unique_block_types,appearance_order=np.unique(self.linearized_block_types,return_index=True)
        #check how many longitudinal and latitudinal blocks there are in each ring layer, from the smartblock name
        ring_layers=[list(map(int,el.split('_')[1:])) for el in unique_block_types[np.argsort(appearance_order)] if 'ringblock' in el]
        n_per_layer=np.prod(ring_layers,axis=1)
        #connect hooks into a square grid
        for idx,(long,lat) in enumerate(ring_layers):
            shift=np.hstack((0,np.cumsum(n_per_layer)))[idx] #each ring's starting hook index
            for i in range(long):
                for j in range(lat):
                    node=(i*lat+j)%n_per_layer[idx]+shift #ring arithmetic
                    if j!=lat-1:
                        node_down=(i*lat+j+1)%n_per_layer[idx]+shift
                        self.connected_hooks.append((node,node_down)) # every j hooks is on a new block at the other edge of the ring. this connection would be diagonal
                    node_side=((i+1)*lat+j)%n_per_layer[idx]+shift
                    self.connected_hooks.append( (node,node_side) )
        #grab indices by block-type
        els=[idx for idx,el in enumerate(self.linearized_block_types) if 'electrode' in el]
        latches=[idx for idx,el in enumerate(self.linearized_block_types) if 'latch' in el]
        pads=[idx for idx,el in enumerate(self.linearized_block_types) if 'pad' in el]
        block_hooks=[idx for idx,el in enumerate(self.linearized_block_types) if 'ringblock' in el]

        latch_inside_hooks=np.asarray([idx for idx,el in enumerate(self.linearized_block_types) if 'latch' in el]) 
        latch_inside_hooks=list(map(tuple,latch_inside_hooks.reshape(len(latch_inside_hooks)//2,2)))


        self.connected_hooks+=who_connects_who_closest(pads,latches,hook_tolerance)
        self.connected_hooks+=who_connects_who_closest(latches,block_hooks,hook_tolerance)
        self.connected_hooks+=who_connects_who_closest(els,block_hooks,hook_tolerance)
        self.connected_hooks+=who_connects_who_closest(els,latches,1) #electrodes should only connect to blocks here

        self.connected_hooks+=latch_inside_hooks
        source=int(1e9)
        drain=int(1e10)
        
        pool_sources=[(source,e) for e in els]
        pool_drain=[(drain,e) for e in pads]
        
        #All the pads are connected to a drain now, and all the electrodes to a source.
        #We can now run a node-disjoint flow algorithm between source and drain to find #electrode non-crossing paths.
        
        dists=np.sqrt(np.sum(np.diff(self.linearized_hook_pos[self.connected_hooks],axis=-2)**2,axis=-1))
        dists=np.hstack((dists.reshape(-1),np.ones(len(pool_drain)),np.ones(len(pool_sources))))
        pairs=self.connected_hooks+pool_drain+pool_sources
        self.graph=nx.from_edgelist(pairs)
        #add pool and drain edge pairs to also set them to capacity 1 below
        capacities=np.hstack( (np.asarray(pairs),dists.reshape(-1,1)+1e-1))

        dict_capa={(int(a[0]),int(a[1])):a[2] for a in capacities}
        nx.set_edge_attributes(self.graph, dict_capa, name='capacity')
        print('Finishing setting up the graph\n')
        all_paths=list(nx.algorithms.connectivity.disjoint_paths.node_disjoint_paths(self.graph,source,drain,flow_func=nx.flow.edmonds_karp))
        self.all_paths=[np.asarray(p)[np.asarray(p)<source] for p in all_paths]
        print('Finished routing the gold leads')
        
        #Make all routed lines into polygons, and into a new device
        self.gold=Device()
        self.gold_paths=[]
        for pidx,p in enumerate(self.all_paths):
            x,y,cum_length_per_latch=construct_path(p,10)

            dx,dy=np.diff(x),np.diff(y)
            cost=(dx[1:]*dx[:-1]+dy[1:]*dy[:-1])/(np.sqrt(dx[1:]**2+dy[1:]**2)*np.sqrt(dx[:-1]**2+dy[:-1]**2))
            kink=np.where(cost<0.5)[0]
            kink=np.hstack((kink))#keep only smooth angles
            chunks=group_consecutives(kink+1)
            padded_chunks=list(map(lambda x: [x[0]-3]+x+[x[-1]+3],chunks)) #smooth out bezier
            curve=[]
            for idx,bit in enumerate(padded_chunks):
                if idx==0:
                    curve.append(np.vstack((x[:bit[0]+1],y[:bit[0]+1])).T)
                nodes = np.asfortranarray([x[bit],y[bit]])
                crv = bezier.Curve(nodes, degree=len(nodes[0])-1)

                x_bez,y_bez= crv.evaluate_multi(np.linspace(0,1,10))
                curve.append(np.vstack((x_bez,y_bez)).T)
                if idx!=len(padded_chunks)-1:
                    next_bit=padded_chunks[idx+1]
                    curve.append(np.vstack((x[bit[-1]:next_bit[0]+1],y[bit[-1]:next_bit[0]+1])).T)
                if idx==len(padded_chunks)-1:
                    curve.append(np.vstack((x[bit[-1]:],y[bit[-1]:])).T)
            
            self.gold_paths.append(np.concatenate(curve))
#           
            curve_length_cum=np.cumsum(np.sqrt(np.sum(np.diff(d.gold_paths[pidx],axis=0)**2,axis=1)))
            
            if len(cum_length_per_latch)==10:
                b1,b2=np.asarray(cum_length_per_latch)[[5,7]]#where to start thickening the gold #end of second and third latch
            elif len(cum_length_per_latch)==8:
                b1,b2=np.asarray(cum_length_per_latch)[[3,5]]#end of first and second latch
            elif len(cum_length_per_latch)==12:
                b1,b2=np.asarray(cum_length_per_latch)[[7,9]]#..
                
            #print(b1,b2)
            first_bout=np.argmin( np.abs(curve_length_cum-b1))/float(len(curve_length_cum)) #inner electrode have longer path
            second_bout=np.argmin(np.abs(curve_length_cum-b2))/float(len(curve_length_cum))
            third_bout = np.argmin(np.abs(curve_length_cum-cum_length_per_latch[-3]))/float(len(curve_length_cum))
            fourth_bout = (np.argmin(np.abs(curve_length_cum-cum_length_per_latch[-2]))-20)/float(len(curve_length_cum))#beg of last latch

            #print(np.argmin( np.abs(curve_length_cum-b1)),np.argmin(np.abs(curve_length_cum-b2)))
            self.gold.add_polygon(thicken_path_modulate(np.concatenate(curve)[:,0],np.concatenate(curve)[:,1],
                                                   amps=[1.5,5,8,4,2.5],percs=[first_bout,second_bout,third_bout,fourth_bout]
                                                        ,steep=[0.001,0.001,0.001,0.005]))
           
           # self.gold.add_polygon(thicken_path(x,y,
           #                                    start_w=1.5,stop_w=gold_thickness/2.,frac=0.2))
        for idx in els:
            self.gold.add_polygon(self.smart_blocks[self.linear_to_block_index[idx]].polygon)
        #for idx in pads:
        #    self.gold.add_polygon(self.smart_blocks[self.linear_to_block_index[idx]].polygon)
        pads=[idx for idx,el in enumerate(self.smart_blocks) if el.block_type=='pad']
        for idx,p in enumerate(pads):
            x,y=self.all_hook_pos[p][0]
            port1=Port(midpoint=(x,y),width=25,orientation=0)
            port2=Port(midpoint=(x+10,y),width=500,orientation=180)
            #self.gold<<pr.route_basic(port1,port2,width_type='sine')
            rect=pg.rectangle((7+13*idx,250)).movex(x+10).movey(y-125)
            self.gold<<rect
            #self.add_smart_block(SmartBlock('pad_to_open',np.asarray(rect.get_polygons())[0],[0]),add_to_design=False)
    def make_top_layer(self):
        '''This takes the polygons of pad_to_open smart blocks, and electrode smart blocks
        and operates and optimized boolean substraction on the first layer'''
        def quick_bool(A,B):
            holes=B.get_polygons()
            A_p=A.get_polygons()
            index_keeper=np.concatenate([np.ones(len(el))*i for i,el in enumerate(A_p)]).astype(int)
            flat_A_p=np.asarray(np.concatenate(A_p))
            aTree=cKDTree(flat_A_p)
            bTree=cKDTree(np.concatenate(holes))
            where=np.concatenate(bTree.query_ball_tree(aTree,r=50))
            all_bool_As=np.unique(index_keeper[where.astype(int)])
            #large_pad_idx=np.where(np.asarray([len(el) for el in A_p])==4)[0][0].astype(int)
            #all_bool_As=np.hstack((all_bool_As,large_pad_idx))
            temp_A=Device()
            for i in all_bool_As:
                temp_A.add_polygon(A_p[int(i)])
            bigpad_idx=np.argmax(np.asarray(list(map(lambda x: np.max(x[:,1]),A_p))))
            temp_A.add_polygon(A_p[bigpad_idx]) #add polygon with largest 'y' <- its the fat pad. it has too few points to be detected as a neighbor
            opened=pg.boolean(temp_A,B,operation='A-B')
            for i in range(len(A_p)):
                if i not in all_bool_As:
                    if i!=bigpad_idx: #dont add the large pad a second time
                        opened.add_polygon(A_p[i])
            return opened
        holes=Device()
        open_pads=[el.polygon for idx,el in enumerate(self.smart_blocks) if el.block_type=='pad_to_open'] 
        els=[el.polygon for idx,el in enumerate(self.smart_blocks) if el.block_type=='electrode'] 
        for o in open_pads:
            holes.add_polygon(open_pads)
        for e in els:
            holes.add_polygon(els)
        self.third_layer=quick_bool(self,holes)

In [8]:
n_loop=512
n_ondulation=[32,32,32,32,16,16]
n_latch=[32,32,32,16,16]

rs=[5000,3000,1500,550,350,200]
r_mod=[0.03,0.03,0.03,0.03,0.05,0.05]
thicknesses=[300,100,50,9,9,9]
polys=[]
mesh_thick=[25,15,9,9,9]

d=SmartDevice()

ring_long_blocks=[512,512,512,512,256,128]
ring_long_lats=[16,6,4,2,2,2] #lateral 1 is different. it makes the blocks very centered around the middle, although both have two up/down branches
## Make the loop-wavey-rings -- IT IS IMPORTANT TO MAKE THESE FIRST.
for idx,(rad,thick) in enumerate(zip(rs,thicknesses)):
    r=rad+r_mod[idx]*rad*np.sin(np.linspace(0,n_ondulation[idx]*2*np.pi,n_loop))
    initial_rot=30
    t=np.linspace(0,2*np.pi,n_loop)+initial_rot
    x,y=r*np.cos(t),r*np.sin(t)
    #lt.plot(x,y)
    poly=thicken_to_poly(x,y,thick)
    if idx==0:
        x_=(np.linspace(0,2*np.pi,n_loop)+initial_rot)%(2*np.pi)
        initial_rot=30
        modul_=2*(np.exp((x_-np.pi/2.)**2/0.25)/(1+np.exp((x_-np.pi/2.)**2/0.25))-0.5)

        t=np.linspace(0,2*np.pi,n_loop)+initial_rot
        x,y=modul_*poly[0,:n_loop]+(1-modul_)*(rad+thick)*np.cos(t),modul_*poly[1,:n_loop]+(1-modul_)*(rad+thick)*np.sin(t)
        #modul mixes smoothly between a circle and the ondulating design to flatten out the top of the outer circle for
        #gold line routing
        poly[0,:n_loop]=x
        poly[1,:n_loop]=y
    polys.append(poly)
    d.add_polygon(poly)
    smart_ringblock_for_routing=chop_ring(poly,n_long=ring_long_blocks[idx],n_lat=ring_long_lats[idx])
    for s in smart_ringblock_for_routing:
        d.add_smart_block(s,add_to_design=False) # we dont want to chop up the ring. this is only for routing gold

## Add the armlets ##
## --------------- ##
## define parameters for armlets for each 4 levels
idx1=[0,0,0,0,0] #where the latch starts
idx2=[63,95,63,60,60] #where the latch ends
ang1=[50,30,10,10,10,10] #how quickly it turns away from normal from the departing
ang2=[80,10,10,10,10,10] #how quickly it turns into normal on the arrival
mag1=[1300,700,100,50,50] #how far it shoots along the normal (controls the bending)
mag2=[1400,700,100,100,100] # how far it shoots along the normal on arrival (controls the bending)
nbezier=[55,45,25,25,25] # how many points to resolve shape : can be played with to smoothen the end of latch
thickness=mesh_thick
npts1=[3,3,3,3,3] # how wide is the latch
npts2=[3,3,3,3,3] # how wide is the latch
## -------------- ##

##6 support winglets on the first layer##
s_idx1=[-3,-5,-7,3,5,7] #where the latch starts
s_idx2=[20,27,35,397,390,383] #where the latch ends
s_ang1=[10,50,10,50,50,10] #how quickly it turns away from normal from the departing
s_ang2=[10,10,50,20,-20,5] #how quickly it turns into normal on the arrival
s_mag1=[300,200,100,150,150,150] # how far it shoots along the normal (controls the bending)
s_mag2=[150,200,300,150,150,150] # how far it shoots along the normal on arrival (controls the bending)
# how many points to resolve shape : can be played with to smoothen the end of latch
winglet_thickness=8
##-------------- ##


latch_collection=[]

for l in range(len(thicknesses)-1):
    for n in range(n_latch[l]):

        inner=np.vstack( (polys[l][:,n_loop:].T,polys[l][:,n_loop:].T )) #from outer ring
        outer=np.vstack( (polys[l+1][:,:n_loop].T,polys[l+1][:,:n_loop].T) ) #to inner ring

        shift=int(n*n_loop/n_latch[l]) #how much to move-over along the ring
        smart_block=smart_latch(inner,outer,idx1=idx1[l]+shift,idx2=idx2[l]-shift,ang_1=ang1[l],
                             ang_2=ang2[l],mag1=mag1[l],mag2=mag2[l],n_bezier=nbezier[l],
                             thickness=thickness[l],n_pts_1=npts1[l],n_pts_2=npts2[l])
#         #add support winglets on first layer
#         if l==0:
#             for w in range(6):
#                 pol,_=snap_between_poly(inner,smart_block.polygon,idx1=s_idx1[w]+shift,idx2=s_idx2[w],ang_1=s_ang1[w]
#                                         ,ang_2=s_ang2[w],mag1=s_mag1[w],mag2=s_mag2[w],thickness=winglet_thickness
#                                         ,n_pts_1=1,n_pts_2=2,n_bezier=15)
#                 d.add_polygon(pol)

        #store latches for making more webs
        latch_collection.append([smart_block.polygon,n,l])
        d.add_smart_block(smart_block)

# #iterative webbing
start_web_frac=[0.88,0.87,0.8,0.7,0.7]
end_web_frac=[0.09,0.16,0.3,0.4,0.4]
n_web = [9,7,7,2,2]
for lidx,j in enumerate(range(len(n_web))):  
    for i in range(n_latch[lidx]):   
        try:
            pol1=[el[0] for el in latch_collection if (el[1]==i) and (el[2]==j)][0]
            pol2=[el[0] for el in latch_collection if (el[1]==(i+1)%n_latch[lidx]) and (el[2]==lidx)][0]
            l1=len(pol1)//2
            l2=len(pol2)//2
            sub_pol=[]
            ##Webbing between latches
            for n in range(n_web[j]):
                frac1=np.linspace(start_web_frac[j],end_web_frac[j],n_web[j])[n]
                frac2=1-frac1 #the polygon runs inverse - we want them to match at the same 'height' along either latch,
                #such that the connecting latch is horizontal when the structure fully unfolds
                mag1,mag2,thickness=100,100,3
                if j>2:
                    mag1,mag2=30,30
                if j>2:
                    thickness=3
                pol,_=snap_between_poly(pol1[l1:,:],pol2[:l2,:],idx1=int(frac1*l1),idx2=int(frac2*l2),mag1=mag1,mag2=mag2,
                                       ang_1=10,ang_2=10,thickness=thickness,n_pts_1=3,n_pts_2=3,n_bezier=25)
                d.add_polygon(pol)
                sub_pol.append(pol)
                ##Webbing between webs
                if j<2: #skip for the innermost
                    if n>0:
                        subpol1=sub_pol[n-1]
                        subpol2=sub_pol[n]
                        sl1=sl2=len(subpol1)//2
                        if j==1: #next inner ring
                            if n<5:
                                for z in range(3):
                                    frac1=[0.2,0.5,0.8][z]
                                    frac2=1-np.linspace(0.3,0.85,3)[z]
                                    subpol,_=snap_between_poly(subpol2[sl1:,:],subpol1[:sl2,:],idx1=int(frac1*sl1),idx2=int(frac2*sl2),mag1=60,mag2=60,
                                                       ang_1=10,ang_2=10,thickness=3,n_pts_1=7,n_pts_2=7,n_bezier=25)
                                    d.add_polygon(subpol)

                            if n>=5:
                                for z in range(2):
                                    frac1=[0.2,0.8][z]
                                    frac2=1-np.linspace(0.3,0.9,2)[z]
                                    subpol,_=snap_between_poly(subpol2[sl1:,:],subpol1[:sl2,:],idx1=int(frac1*sl1),idx2=int(frac2*sl2),mag1=60,mag2=60,
                                                       ang_1=10,ang_2=10,thickness=3,n_pts_1=7,n_pts_2=7,n_bezier=25)
                                    d.add_polygon(subpol)
                        if j==0:
                            for z in range(3):
                                frac1=[0.2,0.5,0.8][z]
                                frac2=1-np.linspace(0.3,0.85,3)[z]
                                subpol,_=snap_between_poly(subpol2[sl1:,:],subpol1[:sl2,:],idx1=int(frac1*sl1),idx2=int(frac2*sl2),mag1=60,mag2=60,
                                                   ang_1=10,ang_2=10,thickness=3,n_pts_1=7,n_pts_2=7,n_bezier=25)
                                d.add_polygon(subpol)

        except:
            print(n_latch[lidx],lidx)
            continue

In [9]:
qp(d)

In [10]:
###Make the connector pad and electrodes
n_pads=32
n_el=32
pad_width=420
pad_block_height=-3000
shift_pads_up=15500
x,y=np.linspace(8450,25405,11*(n_pads+2)),np.ones(11*(n_pads+2))*pad_block_height #make 3 points per pad, and plan for +2 pads to center from edges
bot=np.vstack((y,x))
top=bot.copy()+np.array([420,0]).reshape(2,1)
rect=np.hstack((bot,top[:,::-1]))
d.add_polygon(rect[:,::-1]+np.array([[2790,shift_pads_up]]).T)



port1=pg.Port(width=2800, midpoint=(0,5060), orientation=90)
port2=pg.Port(width=420, midpoint=(0,7500), orientation=270)
port3=pg.Port(width=420, midpoint=(0,9000-550+shift_pads_up), orientation=90)
port4=pg.Port(width=18000, midpoint=(0,pad_block_height), orientation=270)

block1=pr.route_basic(port1,port2,width_type='sine')
block2=pr.route_basic(port2,port3,width_type='sine')

tmp=pg.Device()
tmp.add_polygon(polys[0].T)
d<<pg.boolean(block1,tmp,
              operation='A-B')
d<<block2




for i in range(n_pads):
    pad_x=np.hstack((x[11*(i+1)+5]-pad_width/2,x[11*(i+1)+5],x[11*(i+1)+5]+pad_width/2))
    print(pad_x)
    pad_x=np.hstack((pad_x,pad_x[::-1]))
    pad_y=pad_block_height*np.ones(6)+[0,0,0,2500,2500,2500]
    hook=[-2]
    d.add_smart_block(SmartBlock('pad',np.vstack((pad_y+130+150,pad_x+shift_pads_up)).T,hook),add_to_design=False)
    plt.plot(pad_y+130,pad_x)
    
#put ring_electrodes on each
tmp=pg.Device()
platinum=SmartDevice()
for i in range(n_el):
#     r_shift=[5,-17][i%2]
#     angle_shift=[15.5*np.pi/180.,-23/180.*np.pi][i%2]
#     r=rs[4-i%2]+r_shift # jump from last to second last ring interchangeably
#     angle=2*np.pi*i/n_el+angle_shift

#     x,y=r*np.cos(angle),r*np.sin(angle)
    if i<16:
        shifter=-1-i%2
    else:
        shifter=-3
    rad=rs[shifter]
    r=rad+r_mod[shifter]*rad*np.sin(np.linspace(0,n_ondulation[shifter]*2*np.pi,n_loop))
    initial_rot=30
    t=np.linspace(0,2*np.pi,n_loop)+initial_rot
    ang_shift=[7,13,8][shifter] #shift is important for first entry as inner ring only has 256 chops, and n_loop is 256. The center
    #of the electrode and the block is offset by 1/512. The connection distance is very strict so center of electrode must=
    #block node
    ang=t[ang_shift+n_loop//(n_el//2)*(i%16)]
    the_r=r[ang_shift+n_loop//(n_el//2)*(i%16)]
    x,y=the_r*np.cos(ang),the_r*np.sin(ang)

    plt.plot(x,y,'o')
    d.add_smart_block(smart_electrode(12.5,x,y,resolution=100),add_to_design=False)
    platinum.add_smart_block(smart_electrode(12.5,x,y,resolution=100),add_to_design=True)
    d<<pg.circle(radius=15.5).movex(x).movey(y)


[8967.2922252 9177.2922252 9387.2922252]
[9467.30563003 9677.30563003 9887.30563003]
[ 9967.31903485 10177.31903485 10387.31903485]
[10467.33243968 10677.33243968 10887.33243968]
[10967.3458445 11177.3458445 11387.3458445]
[11467.35924933 11677.35924933 11887.35924933]
[11967.37265416 12177.37265416 12387.37265416]
[12467.38605898 12677.38605898 12887.38605898]
[12967.39946381 13177.39946381 13387.39946381]
[13467.41286863 13677.41286863 13887.41286863]
[13967.42627346 14177.42627346 14387.42627346]
[14467.43967828 14677.43967828 14887.43967828]
[14967.45308311 15177.45308311 15387.45308311]
[15467.46648794 15677.46648794 15887.46648794]
[15967.47989276 16177.47989276 16387.47989276]
[16467.49329759 16677.49329759 16887.49329759]
[16967.50670241 17177.50670241 17387.50670241]
[17467.52010724 17677.52010724 17887.52010724]
[17967.53351206 18177.53351206 18387.53351206]
[18467.54691689 18677.54691689 18887.54691689]
[18967.56032172 19177.56032172 19387.56032172]
[19467.57372654 19677.573

In [11]:
# blocks=[el for el in d.smart_blocks]# if (el.block_type=='ringblock_512_0')or(el.block_type=='ringblock_256_0')or(el.block_type=='ringblock_128_0')]
# for b in blocks:
#     x,y=b.polygon.T
#     plt.plot(x,y)

# qp(d)

In [12]:
qp(d)

In [13]:
top_gold_width=5
top_gold_pitch=13 #center to center

ordered_ports_pad=[]
ordered_ports_to_pad=[]
ordered_ports_to_ring=[]
ordered_ports_ring=[]

wider_gold_device=pg.Device()

for i in range(n_pads):
    x_pad=np.linspace(8450,25405,11*(n_pads+2))+shift_pads_up
    ordered_ports_pad.append(pg.Port(midpoint=(-195,x_pad[11*(i+1)+5]),width=top_gold_width,orientation=0))
    ordered_ports_to_pad.append(pg.Port(midpoint=((-15.5+i)*top_gold_pitch,11000-12*top_gold_pitch),width=top_gold_width,orientation=270))
    
    rim=polys[0]
    lenrim=len(rim[0]) #512 or 1024 etc..
    
    x,y=polys[0][:,int(0.223*lenrim)+i]
    orient=(np.linspace(0,2*np.pi,n_loop)+initial_rot)[int(0.223*lenrim)+i]
    orient=orient*180
    ordered_ports_ring.append(pg.Port(midpoint=(x,y),width=5,orientation=orient-90))
    plt.plot(x,y,'o')
for i in range(n_pads):
    P_top_to_mid=pr.path_L(ordered_ports_pad[i],ordered_ports_to_pad[i])
   # mid_xy=pp.smooth(P_top_to_mid,radius=1).points
    mid_xy=P_top_to_mid.points #add points to avoid smoothing weird stuff
    #print(mid_xy)
    mid_xy=[]
    for z in range(len(P_top_to_mid.points)-1):
        p1=P_top_to_mid.points[z]
        p2=P_top_to_mid.points[z+1]
        t=np.linspace(0,1,150).reshape(-1,1)
        mid_xy.append(p1+(p2-p1)*t)
    mid_xy=np.concatenate(mid_xy)
    #print(mid_xy)
    wider_gold_device<<pr.route_sharp(ordered_ports_pad[i],ordered_ports_to_pad[i],
                                         path_type='L',radius=5,length1=100)
    P_mid_to_ring=pr.path_Z(ordered_ports_to_pad[i],ordered_ports_ring[-i-1],
                           length1=4050)
    wider_gold_device<<pr.route_smooth(ordered_ports_to_pad[i],ordered_ports_ring[-i-1],
                           length1=4050,path_type='Z',radius=3)
    mid_xy2=pp.smooth(P_mid_to_ring,radius=top_gold_width).points
    mid_xy=np.vstack((mid_xy,mid_xy2))[::-1,:]
    smart_line=SmartBlock('latch',polygon=mid_xy,hook_idx=[0,-1],route_line=mid_xy.T)
    d.add_smart_block(smart_line,add_to_design=False)
    
    plt.plot(mid_xy[:,0],mid_xy[:,1],'-o')

In [14]:
# for o in ordered_ports_pad:
#     plt.plot(o.midpoint[0],o.midpoint[1],'o',color='r')
#     print(o.midpoint)
# for o in ordered_ports_to_pad:
#     plt.plot(o.midpoint[0],o.midpoint[1],'o',color='r')
#     print(o.midpoint)
# # for p in d.gold.get_polygons():
# #     plt.plot(p[:,0],p[:,1])


In [15]:
d.route_gold_lines(hook_tolerance=100,el_to_hook_tolerance=50,gold_thickness=20)
d.make_top_layer()
#add the wider gold lines
#d.gold=pg.boolean(d.gold,wider_gold_device,operation='A+B')
d.gold.write_gds('tmp.gds')

Finishing setting up the graph

Finished routing the gold leads


'tmp.gds'

In [17]:
qp(d.gold)

In [18]:
su8=pg.Device()
thin_su8=pg.Device()
t=pg.Device()
width,height,n_rect=120,250,3
thick_su8=6
for o in [ordered_ports_pad[0]]:
    xo,yo=o.midpoint
    frame1=pg.rectangle((width+thick_su8,height+thick_su8)).movex(xo-thick_su8/2.).movey(yo-thick_su8/2.-height/2.)
    frame2=pg.rectangle((width-thick_su8,height-thick_su8)).movex(xo+thick_su8/2.).movey(yo+thick_su8/2.-height/2.)
    frame=pg.boolean(frame1,frame2,operation='A-B')
    su8<<frame
    
    X=pp.CrossSection()

    X.add(width=thick_su8)
    for i in range(-4,24):
        diags=pp.Path()
        diags.points=(np.asarray([ [xo,yo+height/2.-width-i*height/4.],[xo+width,yo+height/2.-i*height/4.]]))
        su8<<diags.extrude(X)
    cut_frame=pg.rectangle( (width+3*thick_su8,2*(height+3*thick_su8/2.))).movex(xo).movey(yo-5*thick_su8/2.-3*height/2.)
    su8=pg.boolean(cut_frame,su8,operation='and')
    mid_bar=pg.rectangle((thick_su8,2*height+2*thick_su8)).movex(xo+(width/2.-thick_su8/2.)).movey(yo-3*thick_su8/2.-3*height/2.)
    su8<<mid_bar
    mid_bar=pg.rectangle((thick_su8,2*height+2*thick_su8)).movex(xo+width-thick_su8/2.).movey(yo-3*thick_su8/2.-3*height/2.)
    su8<<mid_bar
thin_su8=pg.Device()
for i in range(n_pads):
    obj=pg.deepcopy(su8).movey(+500*i)
    thin_su8<<obj
qp(thin_su8)

In [19]:
bottom_gold=pg.Device()
width,height,n_rect=120,250,3
thick_gold=12
for o in [ordered_ports_pad[0]]:
    xo,yo=o.midpoint
    frame1=pg.rectangle((width+thick_gold,height+thick_gold)).movex(xo-thick_gold/2.).movey(yo-thick_gold/2.-height/2.)
    frame2=pg.rectangle((width-thick_gold,height-thick_gold)).movex(xo+thick_gold/2.).movey(yo+thick_gold/2.-height/2.)
    frame=pg.boolean(frame1,frame2,operation='A-B')
    bottom_gold<<frame
    
    X=pp.CrossSection()

    X.add(width=thick_gold)
    for i in range(-4,14):
        diags=pp.Path()
        diags.points=(np.asarray([ [xo,yo+height/2.-width-i*height/4.],[xo+width,yo+height/2.-i*height/4.]]))
        bottom_gold<<diags.extrude(X)
    cut_frame=pg.rectangle( (width+thick_gold,(height+thick_gold))).movex(xo-thick_gold/2.).movey(yo-thick_gold/2.-height/2.)
    bottom_gold=pg.boolean(cut_frame,bottom_gold,operation='and')
    mid_bar=pg.rectangle((thick_gold,height+thick_gold)).movex(xo+(width/2.-thick_gold/2.)).movey(yo-thick_gold/2.-height/2.)
    bottom_gold=pg.boolean(bottom_gold,mid_bar,operation='A+B')

bot_gold=pg.Device()
for i in range(n_pads):
    obj=pg.deepcopy(bottom_gold).movey(+500*i)
    bot_gold<<obj
qp(bot_gold)


In [20]:
bot_gold2=pg.Device()
bot_gold2<<bot_gold
bot_gold2<<pg.deepcopy(bot_gold).movex(-width)
#bot_gold2<<pg.deepcopy(bot_gold).movex(-2*width)

thin_su82=pg.Device()
thin_su82<<thin_su8
thin_su82<<pg.deepcopy(thin_su8).movex(-width)
#thin_su82<<pg.deepcopy(thin_su8).movex(-2*width)


DeviceReference (parent Device "Unnamed", ports [], origin (0, 0), rotation 0, x_reflection False)

In [21]:
new_thin_su8=pg.Device()
for p in thin_su82.get_polygons():
    new_thin_su8.add_polygon(p*np.array([-1,1])-np.array([778-120-255+width,0]))
new_bottom_gold=pg.Device()
for p in bot_gold2.get_polygons():
    new_bottom_gold.add_polygon(p*np.array([-1,1])-np.array([778-120-255+width,0]))
xo,yo=np.min(np.concatenate(new_thin_su8.get_polygons()),axis=0)
yma=np.max(np.concatenate(new_thin_su8.get_polygons()),axis=0)[1]
new_thin_su8<<pg.rectangle( (2*width,thick_su8)).movex(xo).movey(yo)


DeviceReference (parent Device "rectangle", ports [], origin (0, 0), rotation 0, x_reflection False)

In [22]:
t=pg.Device()
t<<d.gold
t<<new_thin_su8
t<<new_bottom_gold
t<<d
qp(t)


In [23]:
d.gold<<new_bottom_gold
tmp=pg.Device()
tmp<<d

DeviceReference (parent Device "Unnamed", ports [], origin (0, 0), rotation 0, x_reflection False)

In [26]:
top_down_order=np.argsort([el[-1][1] for el in d.gold_paths])
d.gold_paths=[d.gold_paths[idx] for idx in top_down_order]
d.all_paths=[d.all_paths[idx] for idx in top_down_order]



In [27]:
top_down_order=np.argsort([el[-1][1] for el in d.gold_paths])
d.gold_paths=[d.gold_paths[idx] for idx in top_down_order]
d.all_paths=[d.all_paths[idx] for idx in top_down_order]
def quaternary (n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 4)
        nums.append(str(r))
    return ''.join(reversed(nums))

barcodes=[]
for idx,p in enumerate(d.all_paths):
    first_latch_idx=p[[idx for (idx,el) in enumerate(d.linearized_block_types[p]) if el=='latch'][0]]
    block_number=d.linear_to_block_index[first_latch_idx]
    midline=d.smart_blocks[block_number].route_line.T
    
    x,y=midline[::-1,:].T
    plt.plot(x,y)
    
    ds=np.cumsum(np.sqrt(np.sum(np.gradient(midline,axis=0)**2,axis=1)))
    n_x,n_y=-get_norm(x,y)
    edge_x,edge_y=x+4.5*n_x,y+4.5*n_y
    x_i,y_i=interpolate.interp1d(ds,edge_x),interpolate.interp1d(ds,edge_y)
    nx_i,ny_i=interpolate.interp1d(ds,n_x),interpolate.interp1d(ds,n_y)
    
    
    encoding=quaternary(idx).zfill(3)
    n_pts_blob=71
    
    for i in range(len(encoding)):
        t=np.linspace(0,1,n_pts_blob)
        s_interp=np.linspace(10*(i+2)+10*i,10*(i+2)+10*(i+1),n_pts_blob)
        if np.sqrt(edge_x**2+edge_y**2)[0]>500: #put the outer ring barcodes a little further up
            s_interp=np.linspace(10*(i+6)+10*i,10*(i+6)+10*(i+1),n_pts_blob)
        if encoding[i]=='0': #left triangle
            #t=-(2*np.abs(t-0.5)-1) #triangle
            t=t
            t[-1]=0
            fac=1
        if encoding[i]=='1': #square for 1
            t=np.ones(len(t))
            t[0]=0
            t[-1]=0
            fac=1
        if encoding[i]=='2': #right triangle
            #t=np.sqrt( -(t-0.5)**2 + 0.5**2 )
            #fac=1.4
            t=t[::-1]
            t[0]=0
            fac=1
        if encoding[i]=='3': #circle
            t=2*np.sqrt((0.5**2-(t-0.5)**2))
            fac=1
        barcode_x=x_i(s_interp)+7*nx_i(s_interp)*t
        barcode_y=y_i(s_interp)+7*ny_i(s_interp)*t
        plt.plot(barcode_x,barcode_y,color='r')
        if np.sum(t)==0:
            print(idx,i)
        
        barcodes.append(np.vstack((barcode_x,barcode_y)))


In [28]:
ps=np.concatenate(d.get_polygons())
close_poly=[]
for b in barcodes:
    beg=np.argmin( np.sum((ps-b[:,-1])**2,axis=1) )
    end=np.argmin( np.sum((ps-b[:,0])**2,axis=1) )
    close_poly.append(ps[beg:end+1,:])

In [29]:
for b,c in zip(barcodes,close_poly):
    d.add_polygon( np.hstack((b,c.T)))

In [30]:
ttmp=pg.Device()
ttmp<<d
ttmp<<d.gold
qp(ttmp)

In [ ]:
# def ternary (n):
#     if n == 0:
#         return '0'
#     nums = []
#     while n:
#         n, r = divmod(n, 3)
#         nums.append(str(r))
#     return ''.join(reversed(nums))


# top_down_order=np.argsort([el[-1][1] for el in d.gold_paths])
# d.gold_paths=[d.gold_paths[idx] for idx in top_down_order]
# fig,ax=plt.subplots(1,1,figsize=(5,5)) 
# ax.set_facecolor('k')
# top_down_order=np.argsort([el[-1][1] for el in d.gold_paths])
# d.gold_paths=[d.gold_paths[idx] for idx in top_down_order]
# for idx,p in enumerate(d.gold_paths):
#     ax.plot(p[:,0],p[:,1],color=(idx/32.,0,0))


# test_t=pg.Device()
# test_t<<d
# for idx,p in enumerate(d.gold_paths):
#     if idx==0:
#         color='r'
#     else:
#         color=(1,1,1)
#     ax.plot(p[:,0],p[:,1],color=color)
#     n_x,n_y=get_norm(p[:,0],p[:,1])
#     dx,dy=np.gradient(p[:,0]),np.gradient(p[:,1])
#     s=np.cumsum(np.sqrt(dx**2+dy**2))
#     cut_idx=np.argmin(np.abs(s-np.array([100,44])[idx%2]))
#     plt.plot([p[cut_idx,0],p[cut_idx,0]+50*n_x[cut_idx]],[p[cut_idx,1],p[cut_idx,1]+50*n_y[cut_idx]])
#     close_to_polygon_edge=p[cut_idx,:]+mesh_thick[0]/2*np.array( [n_x[cut_idx],n_y[cut_idx]])
#     all_coords=np.concatenate(d.get_polygons())
#     intercept_polys=np.argmin(np.sum((all_coords-close_to_polygon_edge)**2,axis=1))
#     bit_of_poly=all_coords[intercept_polys:intercept_polys+100,:]
#     plt.plot(bit_of_poly[:,0],bit_of_poly[:,1])
#     #arc_length_parametrization
#     dx,dy=np.gradient(bit_of_poly[:,0]),np.gradient(bit_of_poly[:,1])
#     s=np.cumsum(np.sqrt(dx**2+dy**2))
#     step=np.max((1,np.argmin( np.abs(s-10))))+1
    
#     #encoding=bin(idx)[2:].zfill(4)
#     encoding=ternary(idx).zfill(3)
#     n_pts_blob=70
#     for i in range(len(encoding)):
#         t=np.linspace(0,1,n_pts_blob)
#         if not idx%2:
#             support_xy=bit_of_poly[step*(i)+2*i:step*(i+1)+2*i] #space them more
#         else:
#             support_xy=bit_of_poly[step*i:step*(i+1)]
#         func=interpolate.interp1d(np.linspace(0,1,step),support_xy,axis=0)
#         #t=np.linspace(0,1,n_pts_blob)
#         curve=func(t) # interpolate the polygon support to more points
#         n_x,n_y=get_norm(curve[:,0],curve[:,1])
        
#         if encoding[::-1][i]=='0': #left triangle
#             #t=-(2*np.abs(t-0.5)-1) #triangle
#             t=t
#             t[-1]=0
#             fac=1
#         if encoding[::-1][i]=='1': #square for 1
#             t=np.ones(len(t))
#             t[0]=0
#             t[-1]=0
#             fac=1
#         if encoding[::-1][i]=='2': #right triangle
#             #t=np.sqrt( -(t-0.5)**2 + 0.5**2 )
#             #fac=1.4
#             t=t[::-1]
#             t[0]=0
#             fac=1
#         length_support=np.sum(np.sqrt(np.sum(np.diff(support_xy,axis=0)**2,axis=1)))
#         #barcode=curve+(5+3*i)*np.exp(-(2*t-1)**power/(0.1)).reshape(-1,1)*np.vstack((n_x,n_y)).T #increasing size for direction
#         barcode=curve+fac*length_support*t.reshape(-1,1)*np.vstack((n_x,n_y)).T
#         plt.plot(barcode[:,0],barcode[:,1])# center gaussian
#         plt.plot(curve[:,0],curve[:,1],color='r')
#         polygon=np.vstack((barcode,support_xy))
#         d.add_polygon(polygon)
#         #d.third_layer.add_polygon(polygon)
#         test_t.add_polygon(polygon)

In [ ]:
qp(d)

In [ ]:
rect

In [ ]:
robe=Device()
def thicken_path(x,y,thick):
    xn,yn=get_norm(x,y)
    xpol=np.hstack( (x+thick*xn,(x-thick*xn)[::-1]))
    ypol=np.hstack( (y+thick*yn,(y-thick*yn)[::-1]))
    return np.vstack((xpol,ypol)).T
for r in np.arange(6000,9500,300):
    squirc_b=squircle(r,x_c=0,y_c=0,p=7)
    sq=thicken_path(squirc_b[0],squirc_b[1],75)
    a=Device()
    b=Device()
    a.add_polygon(sq)
    b.add_polygon(rect+np.array([0,5000]).reshape(2,1))
    robe<<pg.boolean(a,b,operation='A-B')
for idx,r in enumerate(np.arange(5500,6000,300)):
    squirc=squircle(r,x_c=0,y_c=0,p=4+idx)
    robe.add_polygon(thicken_path(squirc[0],squirc[1],75))

pol2=sq #the largest
pol1=polys[0]
l1=len(pol1[0])//2
l2=len(pol2)//2
frac1=np.linspace(0.48,1.465,45)[2:43]
frac2=np.linspace(.75,-0.25,45)[2:43]
for i in range(len(frac1)):
    pol,_=snap_between_poly(pol1.T[:l1,:],pol2[l2:,:],idx1=int(frac1[i]*l1)%l1,idx2=int(frac2[i]*l2)%l2,mag1=80,mag2=80,
                                               ang_1=10,ang_2=10,thickness=140,n_pts_1=3,n_pts_2=15,n_bezier=25)
    robe.add_polygon(pol)
qp(robe)

In [ ]:
# tmp=Device()
# squirc=squircle(9350,x_c=0,y_c=0,p=7)
# tmp.add_polygon(squirc)
# a_block=pg.rectangle((2*9390,15000)).movex(-9390).movey(-20000)

# #cutter=(pg.boolean(tmp,a_block,operation='B-A'))
# #cutter.movey(2500)
# #cutter<<pg.deepcopy(cutter).mirror((1,0)).movey(500)

# cutter_top_line=cutter.polygons[0].polygons[0][:-2].T
# top_line=thicken_path(cutter_top_line[0],cutter_top_line[1],thick=20)
# other_line=thicken_path(cutter_top_line[0],-cutter_top_line[1]+500,thick=20)
# robe=pg.boolean(robe,cutter,operation='A-B')
# robe.add_polygon(top_line)
# robe.add_polygon(other_line)
# qp(robe)


In [25]:
first_layer=Device()
first_layer<<d
first_layer<<robe
third_layer=d.third_layer
third_layer<<robe


NameError: name 'robe' is not defined

In [ ]:
qp(first_layer)

In [183]:
new_bottom_gold.write_gds('spiral_short_l1_vXiao_32ch_v1.gds')
new_thin_su8.write_gds('spiral_short_l2_vXiao_32ch_v1.gds')
first_layer.write_gds('spiral_short_l3_vXiao_32ch_v1.gds')
d.gold.write_gds('spiral_short_l4_vXiao_32ch_v1.gds')
third_layer.write_gds('spiral_short_l5_vXiao_32ch_v1.gds')
platinum.write_gds('spiral_short_l6_vXiao_32ch_v1.gds')

'spiral_short_l6_vXiao_32ch_v1.gds'

In [132]:
qp(d.gold)

In [133]:
n_loop=516
n_ondulation=16

rs=[5000,3000,700,200]
thicknesses=[300,100,50,30]
polys=[]
mesh_thick=25

d=SmartDevice()

ring_long_blocks=[512,512,512,128]
ring_long_lats=[8,2,4,4]
## Make the loop-wavey-rings -- IT IS IMPORTANT TO MAKE THESE FIRST.

for idx,(rad,thick) in enumerate(zip(rs,thicknesses)):
    r=rad+0.05*rad*np.sin(np.linspace(0,n_ondulation*2*np.pi,n_loop))
    initial_rot=30
    t=np.linspace(0,2*np.pi,n_loop)+initial_rot
    x,y=r*np.cos(t),r*np.sin(t)
    #lt.plot(x,y)
    poly=thicken_to_poly(x,y,thick)
    
    if idx==0:
        x_=(np.linspace(0,2*np.pi,n_loop)+initial_rot)%(2*np.pi)
        initial_rot=30
        modul_=2*(np.exp((x_-np.pi/2.)**2/0.25)/(1+np.exp((x_-np.pi/2.)**2/0.25))-0.5)

        t=np.linspace(0,2*np.pi,n_loop)+initial_rot
        x,y=modul_*poly[0,:n_loop]+(1-modul_)*(rad+thick)*np.cos(t),modul_*poly[1,:n_loop]+(1-modul_)*(rad+thick)*np.sin(t)
        #modul mixes smoothly between a circle and the ondulating design to flatten out the top of the outer circle for
        #gold line routing
        poly[0,:n_loop]=x
        poly[1,:n_loop]=y
    polys.append(poly)
    d.add_polygon(poly)
    smart_ringblock_for_routing=chop_ring(poly,n_long=ring_long_blocks[idx],n_lat=ring_long_lats[idx])
    for s in smart_ringblock_for_routing:
        d.add_smart_block(s,add_to_design=False) # we dont want to chop up the ring. this is only for routing gold

## Add the armlets ##
## --------------- ##
## define parameters for armlets for each 3 levels
idx1=[0,0,0] #where the latch starts
idx2=[91,95,40] #where the latch ends
ang1=[50,100,10] #how quickly it turns away from normal from the departing
ang2=[80,10,10] #how quickly it turns into normal on the arrival
mag1=[1300,1500,100] # how far it shoots along the normal (controls the bending)
mag2=[1400,1500,100] # how far it shoots along the normal on arrival (controls the bending)
nbezier=[55,45,25] # how many points to resolve shape : can be played with to smoothen the end of latch
thickness=[mesh_thick,mesh_thick,mesh_thick]
npts1=[3,5,3] # how wide is the latch
npts2=[5,3,7] # how wide is the latch
## -------------- ##

##6 support winglets on the first layer##
s_idx1=[-3,-5,-7,3,5,7] #where the latch starts
s_idx2=[50,75,100,370,340,320] #where the latch ends
s_ang1=[10,10,10,50,50,10] #how quickly it turns away from normal from the departing
s_ang2=[10,10,50,20,-20,5] #how quickly it turns into normal on the arrival
s_mag1=[300,200,100,150,150,150] # how far it shoots along the normal (controls the bending)
s_mag2=[150,200,300,150,350,550] # how far it shoots along the normal on arrival (controls the bending)
# how many points to resolve shape : can be played with to smoothen the end of latch
winglet_thickness=8
##-------------- ##


latch_collection=[]
for n in range(16):
    for l in range(3):
        inner=np.vstack( (polys[l][:,n_loop:].T,polys[l][:,n_loop:].T )) #from outer ring
        outer=np.vstack( (polys[l+1][:,:n_loop].T,polys[l+1][:,:n_loop].T) ) #to inner ring

        shift=int(n*n_loop/16) #how much to move-over along the ring
        smart_block=smart_latch(inner,outer,idx1=idx1[l]+shift,idx2=idx2[l]-shift,ang_1=ang1[l],
                             ang_2=ang2[l],mag1=mag1[l],mag2=mag2[l],n_bezier=nbezier[l],
                             thickness=thickness[l],n_pts_1=npts1[l],n_pts_2=npts2[l])
        #add support winglets on first layer
        if l==0:
            for w in range(6):
                pol,_=snap_between_poly(inner,smart_block.polygon,idx1=s_idx1[w]+shift,idx2=s_idx2[w],ang_1=s_ang1[w]
                                        ,ang_2=s_ang2[w],mag1=s_mag1[w],mag2=s_mag2[w],thickness=winglet_thickness
                                        ,n_pts_1=1,n_pts_2=2,n_bezier=15)
                d.add_polygon(pol)
                
        #store latches for making more webs
        latch_collection.append([smart_block.polygon,n,l])
        d.add_smart_block(smart_block)

#iterative webbing
start_web_frac=[0.48,0.6,0.6]
end_web_frac=[0.20,0.1,0.3]
n_web = [3,5,3]
for j in range(3):  
    for i in range(16):   
        
        pol1=[el[0] for el in latch_collection if (el[1]==i) and (el[2]==j)][0]
        pol2=[el[0] for el in latch_collection if (el[1]==(i+1)%16) and (el[2]==j)][0]
        l1=len(pol1)//2
        l2=len(pol2)//2
        sub_pol=[]
        ##Webbing between latches
        for n in range(n_web[j]):
            frac1=np.linspace(start_web_frac[j],end_web_frac[j],n_web[j])[n]
            frac2=1-frac1 #the polygon runs inverse - we want them to match at the same 'height' along either latch,
            #such that the connecting latch is horizontal when the structure fully unfolds
            pol,_=snap_between_poly(pol1[l1:,:],pol2[:l2,:],idx1=int(frac1*l1),idx2=int(frac2*l2),mag1=100,mag2=100,
                                   ang_1=10,ang_2=10,thickness=8,n_pts_1=3,n_pts_2=3,n_bezier=25)
            d.add_polygon(pol)
            sub_pol.append(pol)
            ##Webbing between webs
            if j<2: #skip for the innermost
                if n>0:
                    subpol1=sub_pol[n-1]
                    subpol2=sub_pol[n]
                    sl1=sl2=len(subpol1)//2
                    if j==0:
                        nz,b,e=2,0.5,0.7
                    else:
                        nz,b,e=3,0.3,0.7
                    for z in range(nz):
                        frac1=np.linspace(b,e,nz)[z]
                        frac2=1-frac1
                        subpol,_=snap_between_poly(subpol2[sl1:,:],subpol1[:sl2,:],idx1=int(frac1*sl1),idx2=int(frac2*sl2),mag1=80,mag2=80,
                                           ang_1=10,ang_2=10,thickness=5,n_pts_1=3,n_pts_2=3,n_bezier=25)
                        d.add_polygon(subpol)

In [196]:
###Make the connector pad and electrodes
n_pads=16
n_el=16
pad_width=500
pad_block_height=8000
x,y=np.linspace(-9000,9000,11*(n_pads+2)),np.ones(11*(n_pads+2))*pad_block_height #make 3 points per pad, and plan for +2 pads to center from edges
bot=np.vstack((x,y))
top=bot.copy()+np.array([0,3000]).reshape(2,1)
rect=np.hstack((bot,top[:,::-1]))
d.add_polygon(rect)

#put pad blocks, and also latches that align to them

idx1s=int(n_loop/4.*2+24)-np.arange(16)*5#+[-2,2,2,-4,
                         # -1,0,0,-1,
                         # 0,0,2,2,
                         # 0,0,2,6]
for i in range(n_pads):
    pad_x=np.hstack((x[11*(i+1)+5]-pad_width/2,x[11*(i+1)+5],x[11*(i+1)+5]+pad_width/2))
    pad_x=np.hstack((pad_x,pad_x[::-1]))
    pad_y=pad_block_height*np.ones(6)+[0,0,0,3000,3000,3000]
    hook=[1]
    d.add_smart_block(SmartBlock('pad',np.vstack((pad_x,pad_y)).T,hook),add_to_design=False)
    
    latch=smart_latch(polys[0][:,:n_loop].T,rect.T,idx1=idx1s[i],idx2=11*(i+1)+5,mag1=200,mag2=1300,ang_1=0,ang_2=0,n_pts_1=3,
               n_pts_2=3,n_bezier=20,thickness=150,n_ondul=15,amp_ondul=80)
    d.add_smart_block(latch)

    
#put ring_electrodes on each
for i in range(n_el):
    r_shift=[15,0][i%2]
    angle_shift=[15.5*np.pi/180.,-13/180.*np.pi][i%2]
    r=rs[3-i%2]+r_shift # jump from last to second last ring interchangeably
    angle=2*np.pi*i/n_el+angle_shift

    x,y=r*np.cos(angle),r*np.sin(angle)
    d.add_smart_block(smart_electrode(12.5,x,y,resolution=20),add_to_design=False)
qp(d)


In [197]:
d.route_gold_lines(gold_thickness=5)
d.make_top_layer()

Finishing setting up the graph

Finished routing the gold leads


In [198]:
d.gold<<pg.rectangle(size=(200,2600)).movex(-8700).movey(8200)
d.gold<<pg.rectangle(size=(200,2600)).movex(-8700+17200).movey(8200)


DeviceReference (parent Device "rectangle", ports [], origin (0, 0), rotation 0, x_reflection False)

In [137]:
left_right_order=np.argsort([el[-1][0] for el in d.gold_paths])
d.gold_paths=[d.gold_paths[idx] for idx in left_right_order]
fig,ax=plt.subplots(1,1,figsize=(5,5))
ax.set_facecolor('k')
left_right_order=np.argsort([el[-1][0] for el in d.gold_paths])
d.gold_paths=[d.gold_paths[idx] for idx in left_right_order]
for idx,p in enumerate(d.gold_paths):
    if idx==0:
        color='r'
    else:
        color=(1,1,1)
    ax.plot(p[:,0],p[:,1],color=color)
    n_x,n_y=get_norm(p[:,0],p[:,1])
    dx,dy=np.gradient(p[:,0]),np.gradient(p[:,1])
    s=np.cumsum(np.sqrt(dx**2+dy**2))
    cut_idx=np.argmin(np.abs(s-np.array([80,30])[(idx+1)%2]))
    plt.plot([p[cut_idx,0],p[cut_idx,0]+50*n_x[cut_idx]],[p[cut_idx,1],p[cut_idx,1]+50*n_y[cut_idx]])
    close_to_polygon_edge=p[cut_idx,:]+mesh_thick/2*np.array( [n_x[cut_idx],n_y[cut_idx]])
    all_coords=np.concatenate(d.get_polygons())
    intercept_polys=np.argmin(np.sum((all_coords-close_to_polygon_edge)**2,axis=1))
    bit_of_poly=all_coords[intercept_polys:intercept_polys+100,:]
    plt.plot(bit_of_poly[:,0],bit_of_poly[:,1])
    #arc_length_parametrization
    dx,dy=np.gradient(bit_of_poly[:,0]),np.gradient(bit_of_poly[:,1])
    s=np.cumsum(np.sqrt(dx**2+dy**2))
    step=np.max((1,np.argmin( np.abs(s-10))))+1
    
    encoding=bin(idx)[2:].zfill(4)
    
    n_pts_blob=40
    for i in range(len(encoding)):
        if encoding[::-1][i]=='0': #rounded gauss for 0
            power=8
        if encoding[::-1][i]=='1': #normal gauss for 1
            power=2
        support_xy=bit_of_poly[step*(i):step*(i+1)]
        func=interpolate.interp1d(np.linspace(0,1,step),support_xy,axis=0)
        t=np.linspace(0,1,n_pts_blob)
        curve=func(t) # interpolate the polygon support to more points
        n_x,n_y=get_norm(curve[:,0],curve[:,1])
        barcode=curve+(5+3*i)*np.exp(-(2*t-1)**power/(0.1)).reshape(-1,1)*np.vstack((n_x,n_y)).T #increasing size for direction
        plt.plot(barcode[:,0],barcode[:,1])# center gaussian
        plt.plot(curve[:,0],curve[:,1],color='r')
        polygon=np.vstack((barcode,support_xy))
        d.add_polygon(polygon)
        d.third_layer.add_polygon(polygon)

In [138]:
first_layer=Device()
first_layer<<d
first_layer<<robe
third_layer=d.third_layer
third_layer<<robe

first_layer.write_gds('spiral_long_l1_v10.gds')
d.gold.write_gds('spiral_long_l2_v10.gds')
third_layer.write_gds('spiral_long_l3_v10.gds')

'spiral_long_l3_v10.gds'

In [21]:
from phidl import Path,CrossSection
from phidl.quickplotter import quickplot as qq
from time import sleep

In [49]:
ctr=0
gold=Device()
for frac in [0.5]:#np.linspace(0,1,20)[1:]:
    t=Device()
    t<<d
    X=CrossSection()
    X.add(width = 25, offset = 0, layer = 2)
    for p in d.gold_paths:
        plen=int(len(p[:,0])*frac)
        x,y=p[:plen,0],p[:plen,1]
        pt=Path()
        pt.points=np.vstack((x,y)).T

        gold<<pt.extrude(X)
    

In [69]:
from matplotlib.colors import hex2color
gold_color=hex2color('#FFD700')

dev_im,_,_,_=draw_device(d)
gld_im,_,_,_=draw_device(gold)
dev_im=np.asarray(dev_im)
gld_im=np.asarray(gld_im)
composite=np.ones((5000,5000,3))*0.5
for z in range(3):
    composite[dev_im>0,z]=0
    composite[gld_im>0,z]=gold_color[z]



In [85]:
def draw_for_growth(a_device):
    a_device
    polys = a_device.get_polygons()

    from PIL import Image, ImageDraw

    #This is messy. could be automated but I was lazy.
    #I upscale the quality and cast it to pixels, because
    #I need a good way to import it into a meshing software for simulation
    #And I do image analysis to reconstruct the mesh..
    size=2000
    img = Image.new('1', (size, size))
    drw = ImageDraw.Draw(img, '1')
    
    scale=8
    
    shift_x,shift_y=1000,700
    
    for p in polys:
        p=np.vstack((p,p[0]))
        #p[:,0]-=np.min(p[:,0])
        #p[:,1]-=np.min(p[:,1])
        
        p=p/scale #scale
        
        #dimx,dimy=int(np.max(p[:,0])),int(np.max(p[:,1]))
        p[:,0]+=shift_x
        p[:,1]+=shift_y

        drw.polygon(list(map(tuple,p)), (1)) 
    del drw

    return img,scale,shift_x,shift_y   

In [108]:
dev_im,_,_,_=draw_for_growth(d)
dev_im=np.asarray(dev_im)
import imageio as io
top_pads=[el for el in d.gold.get_polygons() if (len(el)==198) or (len(el)==4)]

for idx,frac in enumerate(np.linspace(0,1,70)[1:]):
    gold=Device()
    X=CrossSection()
    X.add(width = 25, offset = 0, layer = 2)
    for p in d.gold_paths:
        plen=int(len(p[:,0])*frac)
        x,y=p[:plen,0],p[:plen,1]
        pt=Path()
        pt.points=np.vstack((x,y)).T

        gold<<pt.extrude(X)
    for p in top_pads:
        gold.add_polygon(p)
    gld_im,_,_,_=draw_for_growth(gold)
    gld_im=np.asarray(gld_im)
    composite=np.ones((2000,2000,3))*0.3
    for z in range(3):
        composite[dev_im>0,z]=0
        composite[gld_im>0,z]=gold_color[z]
    io.imsave('gold_video_spiral/'+str(idx).zfill(3)+'.png',composite)

Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

In [72]:
import phidl.path as pp
X1=pr.CrossSection()
X1.add(width=25,offset=0,layer=0,name='wg1')
X2=pr.CrossSection()
X2.add(width=150,offset=0,layer=0,name='wg1')
Xtrans = pp.transition(cross_section1 = X1,
                       cross_section2 = X2,
                       width_type = 'sine')
# Create a Path for the transitional CrossSection to follow
P3 = pp.Path(path=np.vstack((x,y)).T)
q=P3.extrude(Xtrans)
qp(q)

In [76]:
plt.scatter(*np.asarray(q.get_polygons()).T)

In [70]:
X1 = pr.CrossSection()
X1.add(width = 25, offset = 0, layer = 0, name = 'wg')


# Create the second CrossSection that we want to transition to
X2 = pr.CrossSection()
X2.add(width = 15, offset = 0, layer = 0, name = 'wg')

Xtrans = pp.transition(cross_section1 = X1,
                       cross_section2 = X2,
                       width_type = 'sine')
# Create a Path for the transitional CrossSection to follow
P3 = pp.straight(length = 15)
# Use the transitional CrossSection to create a Device
WG_trans = P3.extrude(Xtrans)

qp(WG_trans)


In [353]:
x=np.ones(10000)
t=np.arange(len(x))
frac=0.2
smooth=0.05
offset,div=len(x)*frac,len(x)*smooth
start_w,stop_w=1.5,2.5
transition = 0.5*(np.tanh( (t-offset)/div)+1)*(stop_w-start_w)+start_w

In [354]:
plt.plot(transition)

In [340]:
latches=[idx for idx,el in enumerate(d.linearized_block_types) if 'latch' in el]
pads=[idx for idx,el in enumerate(d.linearized_block_types) if 'pad' in el]
block_hooks=[idx for idx,el in enumerate(d.linearized_block_types) if 'ringblock' in el]

In [359]:
def connect_closest(l1,l2):
    #order matters
    l1=np.asarray(l1)
    l2=np.asarray(l2)
    xy1=d.linearized_hook_pos[l1]
    xy2=d.linearized_hook_pos[l2]

    dists=np.min(cdist(d.linearized_hook_pos[l1],d.linearized_hook_pos[l2]),axis=1)
    who=l2[np.argmin(cdist(d.linearized_hook_pos[l1],d.linearized_hook_pos[l2]),axis=1)]
    #who=who[dists<tol] # distance has to be smaller than 100 microns (avoids pad-latches to connect to ring. later on..fix)
    return list(map(tuple,np.vstack( (l1,who)).T))

In [565]:
x,y=d.linearized_hook_pos.T

In [566]:
plt.scatter(x,y,color='b')
plt.plot(x[np.asarray(d.connected_hooks)].T,y[np.asarray(d.connected_hooks)].T,color='k')

 ...]

In [567]:
els=[idx for idx,el in enumerate(d.linearized_block_types) if el=='electrode']
plt.scatter(x[els],y[els],color='r')


In [568]:
for p in d.all_paths:
    plt.plot(x[np.asarray(p)].T,y[np.asarray(p)].T,color='r')

In [51]:
points=d.gold_paths[0].T

In [52]:
def _compute_segments(points):
    points = np.asfarray(points)
    normals = np.diff(points, axis = 0)
    normals = (normals.T/np.linalg.norm(normals, axis = 1)).T
    dx = np.diff(points[:,0])
    dy = np.diff(points[:,1])
    ds = np.sqrt(dx**2 + dy**2)
    theta = np.degrees(np.arctan2(dy,dx))
    dtheta = np.diff(theta)
    dtheta = dtheta - 360*np.floor((dtheta + 180)/360)
    return points, normals, ds, theta, dtheta

In [55]:
points, normals, ds, theta, dtheta = _compute_segments(points)
colinear_elements = np.concatenate([[False], np.abs(dtheta) < 1e-6, [False]])
if np.any(colinear_elements):
    new_points = points[~colinear_elements,:]
    points, normals, ds, theta, dtheta = _compute_segments(new_points)

if np.any(np.abs(np.abs(dtheta)-180) < 1e-6):
    raise ValueError('[PHIDL] smooth() received points which double-back on themselves' +
    '--turns cannot be computed when going forwards then exactly backwards.')

In [98]:
pp.__file__

'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\phidl\\path.py'

In [91]:
lambda f:pp.euler(num_pts=7200)

<function __main__.<lambda>(f)>

In [96]:
pp.euler.__defaults__[-1]=72000

TypeError: 'tuple' object does not support item assignment

In [101]:
for p in d.gold_paths:
    x,y=p[0]
    print(np.sqrt(x*x+y*y))

209.99995275372035
718.9388819181705
209.99617329242182
718.2094130840662
209.98634892142812
717.4689326103602
209.97048558175635
716.7178882821659
209.94859286632288
715.9567342727961
209.92068401414275
715.1859308691145
209.8867759023234
714.4059441931895
209.84688903585894
713.6172459204184


In [135]:
def tanh_smooth(amps,percs,steep,support):
    x=support
    percs=np.asarray(percs)*len(x)
    steep=np.asarray(steep)*len(x)
    module=np.ones(len(x))*amps[0] # starting thickness
    for idx,(a,p,s) in enumerate(zip(np.diff(amps),percs,steep)):
        
        module+=0.5*(np.tanh( (x-p)/s)+1)*a
    print(percs,steep)
    return module

x=np.linspace(0,10000,10000)

y=tanh_smooth([3,5,10,20],[0.2,0.5,0.7],[0.01,0.01,0.03],x)

[2000. 5000. 7000.] [100. 100. 300.]


In [136]:
plt.plot(y)

In [129]:
plt.plot(np.tanh( (x-0.2*len(x))/(0.01*len(x))))

In [190]:
x,y=d.gold_paths[0].T
c=np.cumsum(np.hstack( (0,np.sqrt(np.sum(np.diff(d.gold_paths[0],axis=0)**2,axis=1)))))
plt.scatter(x,y,c=c)

In [239]:
np.sum(d.linearized_block_types[d.all_paths[4]]=='latch')

10

In [252]:
xy=[]
cum_length_per_latch=[] #useful to thicken gold lines differently
hook_path=d.all_paths[5]
resolution=10
path_by_block=d.linearized_block_types[hook_path]
for i in range(1,len(hook_path)):
    if path_by_block[i]==path_by_block[i-1]=='latch':
        block_number=d.linear_to_block_index[hook_path[i]]
        midline=d.smart_blocks[block_number].route_line.T
        if d.linearized_hook_indices[hook_path[i-1]]>d.linearized_hook_indices[hook_path[i]]: #path is flipped wrt route line
            midline=midline[::-1,:]                                                             
        xy.append(midline)
        cum_length_per_latch.append(   np.sum(np.sqrt(np.sum(np.diff(np.concatenate(xy),axis=0)**2,axis=1))   ))
    else:
        xy_end=d.linearized_hook_pos[hook_path[i]]
        xy_start=d.linearized_hook_pos[hook_path[i-1]]
        dist=np.sum((xy_end-xy_start)**2)
        t=np.linspace(0,1,resolution)
        xy.append((xy_start.reshape(2,1)+t*(xy_end.reshape(2,1)-xy_start.reshape(2,1))).T) #interpolate path with some res.

x,y=np.concatenate(xy).T 
keep=np.where(np.abs(np.diff(x)+np.diff(y))>1e-2)[0]
if len(x)-1 not in keep:
    keep=np.hstack((keep,len(x)-1)) #add back the pad, which is the last element
x,y=x[keep],y[keep]   


In [302]:
for zx in range(16):
    xy=[]
    cum_length_per_latch=[] #useful to thicken gold lines differently
    hook_path=d.all_paths[zx]
    resolution=10
    path_by_block=d.linearized_block_types[hook_path]
    for i in range(1,len(hook_path)):
        if path_by_block[i]==path_by_block[i-1]=='latch':
            block_number=d.linear_to_block_index[hook_path[i]]
            midline=d.smart_blocks[block_number].route_line.T
            if d.linearized_hook_indices[hook_path[i-1]]>d.linearized_hook_indices[hook_path[i]]: #path is flipped wrt route line
                midline=midline[::-1,:]                                                             
            xy.append(midline)
            cum_length_per_latch.append(   np.sum(np.sqrt(np.sum(np.diff(np.concatenate(xy),axis=0)**2,axis=1))   ))
        else:
            xy_end=d.linearized_hook_pos[hook_path[i]]
            xy_start=d.linearized_hook_pos[hook_path[i-1]]
            dist=np.sum((xy_end-xy_start)**2)
            t=np.linspace(0,1,resolution)
            xy.append((xy_start.reshape(2,1)+t*(xy_end.reshape(2,1)-xy_start.reshape(2,1))).T) #interpolate path with some res.

    x,y=np.concatenate(xy).T 
    keep=np.where(np.abs(np.diff(x)+np.diff(y))>1e-2)[0]
    if len(x)-1 not in keep:
        keep=np.hstack((keep,len(x)-1)) #add back the pad, which is the last element
    x,y=x[keep],y[keep]   
    #plt.plot(x,y,'o',color='k')
    x,y=d.gold_paths[zx].T
    #plt.plot(x,y,'*',color='g')
    cumlen_p=np.cumsum(np.sqrt(np.diff(x)**2+np.diff(y)**2))
    minid=np.argmin(np.abs(cumlen_p-cum_length_per_latch[1]))
    #plt.plot(x[minid],y[minid],'o',markersize=15,color='r')
    #tanh_smooth(amps,percs,steep,support):
    if len(cum_length_per_latch)==5:
        a=np.argmin(np.abs(cumlen_p-cum_length_per_latch[0]))
        b

5
4
5
4
5
4
5
4
5
4
5
4
5
4
5
4


In [69]:
x,y=d.gold_paths[1].T

In [71]:
plt.plot(x[:300],y[:300])

In [72]:
P=pp.Path()
P.points=np.vstack( (x[:100][::10],y[:100][::10]) ).T
xs,ys=pp.smooth(P,radius=1.3).points.T

In [73]:
plt.plot(x[:100][::10],y[:100][::10],'o')
plt.plot(xs,ys,color='r',marker='o')

In [104]:
x,y=d.linearized_hook_pos.T
plt.scatter(x,y)